In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping


# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


In [ ]:
f_samples_optimal_clustering = open('samples_optimal_clustering.txt', 'r')
samples_optimal_clustering = np.loadtxt(f_samples_optimal_clustering)
print(samples_optimal_clustering)
f_sample_grid_d1 = open('sample_grid_d1.txt', 'r')
#sample_grid_d1 = 2*(np.loadtxt(f_sample_grid_d1) - 0.5)
sample_grid_d1 = np.loadtxt(f_sample_grid_d1)
print(sample_grid_d1)
f_sample_grid_d2 = open('sample_grid_d2.txt', 'r')
#sample_grid_d2 = 10*(np.loadtxt(f_sample_grid_d2) - 0.1)
sample_grid_d2 = np.loadtxt(f_sample_grid_d2)
print(sample_grid_d2)
f_lcs0 = open('LocROM_cluster_snap0.txt', 'r')
lcs0 = np.loadtxt(f_lcs0)
f_lcs1 = open('LocROM_cluster_snap1.txt', 'r')
lcs1 = np.loadtxt(f_lcs1)
f_lcs2 = open('LocROM_cluster_snap2.txt', 'r')
lcs2 = np.loadtxt(f_lcs2)
f_lcs3 = open('LocROM_cluster_snap3.txt', 'r')
lcs3 = np.loadtxt(f_lcs3)
f_lcs4 = open('LocROM_cluster_snap4.txt', 'r')
lcs4 = np.loadtxt(f_lcs4)
f_lcs5 = open('LocROM_cluster_snap5.txt', 'r')
lcs5 = np.loadtxt(f_lcs5)
f_lcs6 = open('LocROM_cluster_snap6.txt', 'r')
lcs6 = np.loadtxt(f_lcs6)
f_lcs7 = open('LocROM_cluster_snap7.txt', 'r')
lcs7 = np.loadtxt(f_lcs7)
print(lcs0.shape)

In [ ]:
print(samples_optimal_clustering.shape)
print(sample_grid_d1.shape)
print(sample_grid_d2.shape)


In [ ]:
#train_values = samples_optimal_clustering.reshape(110)
train_values = np.concatenate((lcs0, lcs1, lcs2, lcs3, lcs4, lcs5, lcs6, lcs7))
#print(train_values)
train_values = np.reshape(train_values,(lcs0.shape[0],8), order='F')
print(train_values.shape)
train_values = 1./train_values
np.argmax(train_values, axis = 1)
total_sum = np.max(train_values)
print(total_sum)
for i in range(0,train_values.shape[0]):
    for j in range(0, train_values.shape[1]):
        if train_values[i,j] > 1e+3:
            train_values[i,j] = 1e+3
    sum_row = np.sum(train_values[i,:])
    train_values[i,:] = train_values[i,:] / sum_row
    #max_row = np.max(train_values[i,:])
    #train_values[i,:] = train_values[i,:] / 1e+3
    #train_values[i,:] = lcs0.shape[0] * train_values[i,:] / total_sum
    #print(sum_row)
print(train_values)

In [ ]:
f_sample_grid = open('sample_grid.txt', 'r')
sample_grid = np.loadtxt(f_sample_grid)
print(sample_grid.shape)
#print(sample_grid)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=[2]),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(8)
])

In [ ]:
es = EarlyStopping(monitor='mean_squared_error', min_delta = 0, patience = 1000, mode='min',verbose=1, restore_best_weights = 1)
sgd = keras.optimizers.SGD(lr=0.3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='mse',
              metrics=['mse'])

In [ ]:
model.fit(sample_grid, train_values, epochs=20000, callbacks=[es])

In [ ]:
pred_sg = model.predict(sample_grid)
print(pred_sg.shape)

In [ ]:
np.argmax(pred_sg, axis=1)


In [ ]:
np.argmax(train_values, axis = 1)

In [ ]:
np.argmax(pred_sg, axis=1) - np.argmax(train_values, axis = 1)

In [ ]:
f_fine_sample_grid = open('fine_sample_grid.txt', 'r')
fine_sample_grid = np.loadtxt(f_fine_sample_grid)
print(fine_sample_grid.shape)

In [ ]:
pred_fsg = model.predict(fine_sample_grid)
np.set_printoptions(threshold=np.inf)
#np.argmax(pred_fsg, axis=1)

In [ ]:
f_combination_optimal_c = open('combination_optimal_c.txt', 'r')
combination_optimal_c = np.loadtxt(f_combination_optimal_c)
#print(combination_optimal_c)
#print(sample_grid.shape)
#print(train_values.shape)
#print(combination_optimal_c.shape)
#detailed_train_values = combination_optimal_c.reshape(1640)
#print(detailed_train_values.shape)

In [ ]:
#model_2 = keras.Sequential([
#    keras.layers.Dense(2, activation='relu'),
#    keras.layers.Dense(1024, activation='relu'),
#    keras.layers.Dense(1024, activation='relu'),
#    keras.layers.Dense(8, activation='softmax')
#])
#model_2.compile(optimizer='adam',
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])
#model_2.fit(fine_sample_grid, detailed_train_values, epochs=500)

In [ ]:
#pred_mod2 = model_2.predict(fine_sample_grid)
#save_pred_mod2_fsg = np.argmax(pred_mod2, axis=1)
#np.savetxt('save_pred_mod2_fsg.txt', save_pred_mod2_fsg)

In [ ]:
pred_mod1 = np.argmax(pred_fsg, axis=1)
np.savetxt('test_opti_smart_save_pred_mod1_fsg.txt', pred_mod1)
pred_mod1 = np.reshape(pred_mod1,combination_optimal_c.shape, order='C')
#print(pred_mod1)
#print(combination_optimal_c)
#print(pred_mod1 - combination_optimal_c)
np.count_nonzero(pred_mod1 - combination_optimal_c)
